做时间窗口平移实现数据增强

其实主要的改动就是在伟雁的特征处理上的计算时间加一个shift

+5表示在当前时间+5分钟做时间间隔，如7:55-9:55 当作 8:00-10:00 来做数据增强

In [1]:
import math
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
names=locals()

In [2]:
link_time={}
time_train={}
time_predict={}
time_check={}
test_time_train={}
test_time_check={}
weathers={}
rainingTotalTime={}

# shifted time, +5min or -5min
# +5 表示当前的时间+5分钟来算间隔
# 比如 7:55-9:55 当作 8:00-10:00 来做数据增强
shifted_time = -5

In [3]:
#构造各个link的travel time的dictionary
#有三层，第一层是link(100,101,102...)，第二层是日期[time.month*31+time.day] 月乘31+day的好处是，可以用一个数字来表达这个日期，方便后续操作
#而且从这个数字又可以返回日期 float(str(l//31)+'.'+str(l%31)，第三层是hour，最后存的是，不同link，在不同日期里的不同小时的travel time（存的是具体的数据，后续取平均就能得到小时的平均）
# file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

file_path='/Users/vonderland/学习/hkust/2020Spring/5002/proj/data5002/phase1_training/trajectories_training_phase1_table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

from datetime import timedelta

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    
    trace_start_time += timedelta(minutes=shifted_time)
    
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    if(time.hour in [7,16]):
        try:
            #依序解析該筆紀錄於各link所花費的時間
            for each_seq in travel_seq.split(';'):
                each_seq=each_seq.split('#')
                link_id=each_seq[0]
                start_datetime=each_seq[1]
                start_datetime=datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
                use_seconds=float(each_seq[2])
                if(time.month*31+time.day not in link_time[link_id]):
                    link_time[link_id][time.month*31+time.day]={}
                if(time.hour not in link_time[link_id][time.month*31+time.day]):
                    link_time[link_id][time.month*31+time.day][time.hour]=[]
                    
                link_time[link_id][time.month*31+time.day][time.hour].append(use_seconds)
        except:pass
                

In [4]:
date_key=link_time['100'].keys()

In [5]:
date_key

dict_keys([236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327])

In [6]:

#算label的小时平均时间
names=locals()
for route in travel_times.keys():
    names[str(route+'_'+'8_9')]=[]
    names[str(route+'_'+'9_10')]=[]
    names[str(route+'_'+'17_18')]=[]
    names[str(route+'_'+'18_19')]=[]
    
    

    
for route in travel_times.keys():
        route_time_windows = list(travel_times[route].keys())
        route_time_windows.sort()
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=20)
            tt_set = travel_times[route][time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            
            if time_window_start.hour==8:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'8_9')].append(outline)
            
            if time_window_start.hour==9:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'9_10')].append(outline)
            
            if time_window_start.hour==17:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'17_18')].append(outline)
            
            if time_window_start.hour==18:
                outline = [route,time_window_start.month*31+time_window_start.day,time_window_start,str(avg_tt)]
                names[str(route+'_'+'18_19')].append(outline)
                

In [7]:
#构造label的list
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
time_dict=['8_9','9_10','17_18','18_19']

for i in route_dict:
    for j in time_dict:
        print([i,j])
        print('missing date')
        names[str(i)+'_'+str(j)+'_'+'date']=[]
        for k in names[str(i)+'_'+str(j)]:
            names[str(i)+'_'+str(j)+'_'+'date'].append(k[1])
        names[str(i)+'_'+str(j)+'_'+'date']=list(dict.fromkeys(names[str(i)+'_'+str(j)+'_'+'date']))
        
        kk=0
        for l in date_key:
            if l not in names[str(i)+'_'+str(j)+'_'+'date']:
                print(float(str(l//31)+'.'+str(l%31)))
                kk+=1
        print('len:'+str(kk))


        names[str(i)+'_'+str(j)+'_'+'hr_avg']=[]
        for k in names[str(i)+'_'+str(j)+'_'+'date']:
            temp=[]
            temp1=0
            for m in names[str(i)+'_'+str(j)]:
                if m[1]==k:
                    temp.append(m)
                    temp1+=float(m[3])
            names[str(i)+'_'+str(j)+'_'+'hr_avg'].append([temp1/len(temp),i,temp[0][1],str(temp[0][2].month)+'-'+str(temp[0][2].day),temp[0][2].hour,temp[0][2].weekday()])
        print('\n')

['A_2', '8_9']
missing date
len:0


['A_2', '9_10']
missing date
len:0


['A_2', '17_18']
missing date
8.8
len:1


['A_2', '18_19']
missing date
len:0


['A_3', '8_9']
missing date
len:0


['A_3', '9_10']
missing date
7.21
len:1


['A_3', '17_18']
missing date
8.8
len:1


['A_3', '18_19']
missing date
9.4
len:1


['B_1', '8_9']
missing date
7.22
7.26
8.1
8.16
8.19
8.23
8.26
8.29
8.3
9.0
len:10


['B_1', '9_10']
missing date
8.1
8.13
8.2
8.24
8.26
8.28
len:6


['B_1', '17_18']
missing date
7.19
7.2
7.3
8.4
8.7
8.8
8.13
8.14
8.15
8.18
8.2
8.21
8.22
8.23
8.25
8.26
8.27
8.28
8.29
9.2
9.3
9.4
len:22


['B_1', '18_19']
missing date
7.2
7.21
7.27
7.28
7.3
8.0
8.2
8.3
8.5
8.9
8.12
8.13
8.18
8.19
8.22
8.23
8.25
8.26
8.29
9.0
9.7
9.28
len:22


['B_3', '8_9']
missing date
len:0


['B_3', '9_10']
missing date
7.21
len:1


['B_3', '17_18']
missing date
8.8
len:1


['B_3', '18_19']
missing date
9.4
len:1


['C_1', '8_9']
missing date
8.15
8.16
8.18
8.24
len:4


['C_1', '9_10']
missing date
8.14
8.16

In [8]:
names[str(i)+'_'+str('8_9')+'_'+'hr_avg']

[[132.05, 'C_3', 236, '7-19', 8, 1],
 [251.14499999999998, 'C_3', 237, '7-20', 8, 2],
 [166.15, 'C_3', 238, '7-21', 8, 3],
 [175.385, 'C_3', 240, '7-23', 8, 5],
 [204.33999999999997, 'C_3', 241, '7-24', 8, 6],
 [266.68, 'C_3', 242, '7-25', 8, 0],
 [197.34666666666666, 'C_3', 243, '7-26', 8, 1],
 [223.8, 'C_3', 244, '7-27', 8, 2],
 [294.46000000000004, 'C_3', 245, '7-28', 8, 3],
 [129.59, 'C_3', 246, '7-29', 8, 4],
 [191.78, 'C_3', 247, '7-30', 8, 5],
 [197.7, 'C_3', 248, '7-31', 8, 6],
 [350.05, 'C_3', 249, '8-1', 8, 0],
 [289.14, 'C_3', 250, '8-2', 8, 1],
 [238.65, 'C_3', 251, '8-3', 8, 2],
 [253.25, 'C_3', 252, '8-4', 8, 3],
 [352.295, 'C_3', 253, '8-5', 8, 4],
 [166.20499999999998, 'C_3', 255, '8-7', 8, 6],
 [219.91500000000002, 'C_3', 256, '8-8', 8, 0],
 [198.88333333333333, 'C_3', 257, '8-9', 8, 1],
 [217.82, 'C_3', 258, '8-10', 8, 2],
 [218.47500000000002, 'C_3', 261, '8-13', 8, 5],
 [242.25, 'C_3', 262, '8-14', 8, 6],
 [315.09, 'C_3', 263, '8-15', 8, 0],
 [371.96, 'C_3', 265, '8

In [9]:
link_route1={}
link_route1['A_2']=['110','123','107','108','120','117']
link_route1['A_3']=['110','123','107','108','119','114','118','122']
link_route1['B_1']=['105','100','111','103','116','101','121','106','113']
link_route1['B_3']=['105','100','111','103','122']
link_route1['C_1']=['115','102','109','104','112','111','103','116','101','121','106','113']
link_route1['C_3']=['115','102','109','104','112','111','103','122']

In [10]:
#把前一个小时各个link的average travel time 加入list里，如果有空值，用'null'代替（早上）
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
for i in route_dict:
    temp_link=link_route1[i]
    for j in ['8_9','9_10']:
        names[str(i)+'_'+str(j)+'_'+'feature']=[]
        temp=names[str(i)+'_'+str(j)+'_'+'hr_avg']
        
        for k in temp:
            temp_row=k
            temp_date=k[2]
            
            for l in temp_link:
                if temp_date in link_time[l]:
                    if 7 in link_time[l][temp_date]:
                        temp2=np.mean(link_time[l][temp_date][7])
                    else: temp2='null'
                if temp_date not in link_time[l]: temp2='null'
            
                temp_row.append(temp2)
            names[str(i)+'_'+str(j)+'_'+'feature'].append(temp_row)

In [11]:
#把前一个小时各个link的average travel time 加入list里，如果有空值，用'null'代替（下午）
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
for i in route_dict:
    temp_link=link_route1[i]
    for j in ['17_18','18_19']:
        names[str(i)+'_'+str(j)+'_'+'feature']=[]
        temp=names[str(i)+'_'+str(j)+'_'+'hr_avg']
        
        for k in temp:
            temp_row=k
            temp_date=k[2]
            
            for l in temp_link:
                if temp_date in link_time[l]:
                    if 16 in link_time[l][temp_date]:
                        temp2=np.mean(link_time[l][temp_date][16])
                    else: temp2='null'
                if temp_date not in link_time[l]: temp2='null'
            
                temp_row.append(temp2)
            names[str(i)+'_'+str(j)+'_'+'feature'].append(temp_row)
        

In [12]:
#把同一条路四个小时的信息合并
route_dict=['A_2','A_3','B_1','B_3','C_1','C_3']
time_dict=['8_9','9_10','17_18','18_19']

for i in route_dict:
    names[str(i)+'_'+'whole']=[]
    for j in time_dict:
        temp=names[str(i)+'_'+str(j)+'_'+'feature']
        for k in temp:
            names[str(i)+'_'+'whole'].append(k)
        

In [13]:
#convert to dataframe
import pandas as pd
for i in route_dict:
    temp_link=link_route1[i]
    feature=['label','route','calculated_day','date','hr','weekday']
    for j in temp_link:feature.append(j)
    names[str(i)+'_df']=pd.DataFrame(names[str(i)+'_'+'whole'], columns =feature) 

In [14]:
#找null value

for i in route_dict:
    print(i)
    result = names[str(i)+'_'+'df'].isin(['null'])

    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)

    print('Names of columns which contains null:', columnNames)
    
    names[str(i)+'null_element']=[]
    names[str(i)+'null_row']=[]
    # Iterate over each column and fetch the rows number where
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            names[str(i)+'null_row'].append(row)
            names[str(i)+'null_element'].append([row,col])
    print('index of rows which contains null:',set(names[str(i)+'null_row']))
    print('elements that are null:')
    print(sorted(names[str(i)+'null_element'], key = lambda x: x[0]))
   
    names[str(i)+'null_row']=list(dict.fromkeys(names[str(i)+'null_row']))
    
    print('#of rows that include null:',len(names[str(i)+'null_row']))
    print('\n')

A_2
Names of columns which contains null: []
index of rows which contains null: set()
elements that are null:
[]
#of rows that include null: 0


A_3
Names of columns which contains null: ['119', '114', '118']
index of rows which contains null: {186, 267}
elements that are null:
[[186, '119'], [186, '114'], [186, '118'], [267, '119'], [267, '114'], [267, '118']]
#of rows that include null: 2


B_1
Names of columns which contains null: ['105', '100', '111', '103', '116', '101', '121', '106', '113']
index of rows which contains null: {216, 170, 107}
elements that are null:
[[107, '116'], [107, '101'], [107, '121'], [107, '106'], [107, '113'], [170, '116'], [170, '101'], [170, '121'], [170, '106'], [170, '113'], [216, '105'], [216, '100'], [216, '111'], [216, '103'], [216, '116'], [216, '101'], [216, '121'], [216, '106'], [216, '113']]
#of rows that include null: 3


B_3
Names of columns which contains null: ['105', '100', '111', '103']
index of rows which contains null: {261}
elements tha

In [15]:
#remove null value
for i in route_dict:
    names[str(i)+'_df']=names[str(i)+'_df'].drop(names[str(i)+'null_row'])


In [16]:
#add weather 
# file_path= '/Users/vayne/Desktop/dm_pro_engin/phase1_training/weather.csv'
file_path= '/Users/vonderland/学习/hkust/2020Spring/5002/proj/data5002/weather/weather_July_01_Oct_17_table7.csv'

fr = open(file_path, 'r')
fr.readline()  # skip the header
vol_data = fr.readlines()

for i in range(len(vol_data)):
    each_pass = vol_data[i].replace('"', '').split(',')
    precipitation = float(each_pass[8])
    hour=int(each_pass[1])
    time_window = each_pass[0]
    time_window = datetime.strptime(time_window, "%Y-%m-%d")
    time_window_minute = int(math.floor(time_window.minute / 20) * 20)
    time = datetime(time_window.year, time_window.month, time_window.day, time_window.hour, time_window_minute, 0)
    if time.month*31+time.day not in weathers:
        weathers[time.month*31+time.day] = {}
    for h in range(3):
        weathers[time.month*31+time.day][hour+h]=precipitation
        
#2016.10.10 data missing let it be 0 anytime
weathers[320]={}
for i in range(24):
    weathers[320][i]=0

In [17]:
for i in route_dict:
    temp1=list(names[str(i)+'_df']['calculated_day'])
    temp2=list(names[str(i)+'_df']['hr'])
    weather_list=[]
    for j in range(len(temp1)):
        weather_list.append(weathers[temp1[j]][temp2[j]])
    names[str(i)+'_df']['precipitation']= weather_list

In [18]:
#check the data in the dataframe
for i in route_dict:
    print(i)
    print('length:'+str(len(names[str(i)+'_df'])))
    print('\n')
    print('weekday distribution')
    temp=list(names[str(i)+'_df']['weekday'])
    for j in range(0,7):
        print([temp.count(j),temp.count(j)/len(temp)])
    print('\n')
    print('hour distribution')
    for j in [8,9,17,18]:
        temp=list(names[str(i)+'_df']['hr'])
        print([temp.count(j),temp.count(j)/len(temp)])
    print('\n')

A_2
length:323


weekday distribution
[47, 0.14551083591331268]
[48, 0.14860681114551083]
[48, 0.14860681114551083]
[44, 0.13622291021671826]
[44, 0.13622291021671826]
[44, 0.13622291021671826]
[48, 0.14860681114551083]


hour distribution
[81, 0.25077399380804954]
[81, 0.25077399380804954]
[80, 0.2476780185758514]
[81, 0.25077399380804954]


A_3
length:319


weekday distribution
[47, 0.14733542319749215]
[48, 0.15047021943573669]
[48, 0.15047021943573669]
[43, 0.13479623824451412]
[44, 0.13793103448275862]
[44, 0.13793103448275862]
[45, 0.14106583072100312]


hour distribution
[81, 0.25391849529780564]
[80, 0.2507836990595611]
[79, 0.2476489028213166]
[79, 0.2476489028213166]


B_1
length:261


weekday distribution
[39, 0.14942528735632185]
[38, 0.14559386973180077]
[37, 0.1417624521072797]
[37, 0.1417624521072797]
[34, 0.13026819923371646]
[35, 0.13409961685823754]
[41, 0.15708812260536398]


hour distribution
[71, 0.2720306513409962]
[74, 0.2835249042145594]
[58, 0.2222222222222222]

In [19]:
#output the dataframe
for i in route_dict:
    names[str(i)+'_df'].to_csv("train_shift/" + str(shifted_time) + "min_shifted_" + str(i)+'.csv')


In [20]:
names[str(i)+'_df']

,label,route,calculated_day,date,hr,weekday,115,102,109,104,112,111,103,122,precipitation
0,132.050000,C_3,236,7-19,8,1,8.965,12.305,13.025,29.7675,30.0825,53.212500,2.496667,30.332381,0.0
1,251.145000,C_3,237,7-20,8,2,15.9943,11.1486,11.0543,27.7257,20.2629,36.995714,11.203810,60.224091,0.0
2,166.150000,C_3,238,7-21,8,3,9.85333,11.21,11.4667,25.16,15.0667,29.902500,13.119375,44.910000,0.0
3,175.385000,C_3,240,7-23,8,5,18.1967,12.41,12.4317,27.1967,19.4633,26.613158,4.970526,45.587857,0.0
4,204.340000,C_3,241,7-24,8,6,15.28,13.2,13.496,26.024,20.286,34.165263,9.098421,32.792857,0.0
5,266.680000,C_3,242,7-25,8,0,9.9775,11.5038,12,25.3562,28.2287,52.546800,10.281600,30.988077,0.0
6,197.346667,C_3,243,7-26,8,1,7.90667,8.15667,8.1,21.39,14.8367,38.282000,3.115333,31.829474,0.0
7,223.800000,C_3,244,7-27,8,2,9.035,11.2875,11.9775,25.75,18.495,24.584615,10.490000,34.045556,0.0
8,294.460000,C_3,245,7-28,8,3,7.5575,8.7825,9.04,22.4275,21.98,25.929091,3.207273,59.984167,0.0
9,129.590000,C_3,246,7-29,8,4,11.15,12.255,13.37,32.54,25.97,25.545385,19.913846,80.438750,0.0
